Objective: Practice adding callbacks to Dash apps.

Task:
(1) Build an app that contains the following components user the gapminder dataset: `gdp_pcap.csv`. 

TASK 1 is the same as ASSIGNMENT 4. You are welcome to update your code. 

UI Components:
A dropdown menu that allows the user to select `country`
- The dropdown should allow the user to select multiple countries
- The options should populate from the dataset (not be hard-coded)
A slider that allows the user to select `year`
- The slider should allow the user to select a range of years
- The range should be from the minimum year in the dataset to the maximum year in the dataset
A graph that displays the `gdpPercap` for the selected countries over the selected years
- The graph should display the gdpPercap for each country as a line
- Each country should have a unique color
- The graph should have a title and axis labels in reader friendly format




(2) Write Callback functions for the slider and dropdown to interact with the graph

This means that when a user updates a widget the graph should update accordingly.
The widgets should be independent of each other. 


Layout:
- Use a stylesheet
- There should be a title at the top of the page
- There should be a description of the data and app below the title (3-5 sentences)
- The dropdown and slider should be side by side above the graph and take up the full width of the page
- The graph should be below the dropdown and slider and take up the full width of the page


Submission:
- Deploy your app on Render. 
- In Canvas, submit the URL to your public Github Repo (made specifically for this assignment)
- The readme in your GitHub repo should contain the URL to your Render page. 


**For help you may use the web resources and pandas documentation. No co-pilot or ChatGPT.**
​


In [1]:
# importing packages 
from dash import Dash, dcc, html, Input, Output, callback  
import plotly.express as px
import pandas as pd 

In [2]:
# reading in csv
data = pd.read_csv("/Users/sophiekim/Desktop/4th year (spring)/DS 4003/A4_student/gdp_pcap.csv")

# taking a look at it 
data.head()

,country,1800,1801,1802,1803,1804,1805,1806,1807,1808,...,2091,2092,2093,2094,2095,2096,2097,2098,2099,2100
0,Afghanistan,599,599,599,599,599,599,599,599,599,...,4800,4910,5030,5150,5270,5390,5520,5650,5780,5920
1,Angola,465,466,469,471,472,475,477,479,481,...,24.8k,25.3k,25.9k,26.4k,26.9k,27.4k,28k,28.5k,29.1k,29.6k
2,Albania,585,587,588,590,592,593,595,597,598,...,54k,54.6k,55.2k,55.8k,56.4k,56.9k,57.5k,58.1k,58.7k,59.2k
3,Andorra,1710,1710,1710,1720,1720,1720,1730,1730,1730,...,79.3k,79.5k,79.8k,80.1k,80.4k,80.7k,81k,81.2k,81.5k,81.8k
4,UAE,1420,1430,1430,1440,1450,1450,1460,1460,1470,...,92.5k,92.6k,92.6k,92.7k,92.8k,92.9k,92.9k,93k,93.1k,93.1k


In [3]:
# using panadas melt function to try to get the columns to be country, year, and gdp per capita
# so it is easier to code the app
data_fixed = pd.melt(data, id_vars='country', var_name='year', value_name='gdp_per_capita')

data_fixed.head()

,country,year,gdp_per_capita
0,Afghanistan,1800,599
1,Angola,1800,465
2,Albania,1800,585
3,Andorra,1800,1710
4,UAE,1800,1420


In [4]:
data_fixed.info()
# looks like gdp_per_capita & year are object...should be int or float 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58695 entries, 0 to 58694
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   country         58695 non-null  object
 1   year            58695 non-null  object
 2   gdp_per_capita  58695 non-null  object
dtypes: object(3)
memory usage: 1.3+ MB


In [4]:
# converting year and gdp_per_capita columns to integers
# year first...
data_fixed['year'] = data_fixed['year'].astype(int)

# gdp_per_capita has some values that are k instead of 1,0000 
# getting the values that have the 'k' at the end then converting them to floats (bc demicals)
data_fixed.loc[data_fixed['gdp_per_capita'].str.endswith('k', na = False), 'gdp_per_capita'] = data_fixed.loc[data_fixed['gdp_per_capita'].str.endswith('k', na = False), 'gdp_per_capita'].str.rstrip('k').astype(float) * 1000
# now converting the rest of the gdp_per_capita to floats 
data_fixed['gdp_per_capita'] = data_fixed['gdp_per_capita'].astype(float) 

In [6]:
data_fixed.info()
# yay worked! 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58695 entries, 0 to 58694
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   country         58695 non-null  object 
 1   year            58695 non-null  int64  
 2   gdp_per_capita  58695 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 1.3+ MB


In [5]:
# loading in stylesheet
stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css'] 

# initializing app 
app = Dash(__name__, external_stylesheets=stylesheets)
server = app.server

In [6]:
# making filtered df of unique years for slider marks 
# have to make into integers so that the filtered_years works (to be divisble by 100)
unique_years = sorted([int(year) for year in data_fixed['year'].unique()])

# then making list of years by 100 by saying that put the year in this list if it is divisible by 100 
filtered_years = [year for year in unique_years if year % 100 == 0]

# creating dictionary of the diff years 
slider_marks = {year: str(year) for year in filtered_years}

In [7]:
# building app 
app.layout = html.Div([
    html.H1(children = "Gapminder - GDP per capita from 1800-2100"), # title   (below is the paragraph -- in the second div)
    html.Div(children = "This dataset is from Gapminder's estimates of GDP per capita for almost all countries in the world from 18000 to 2100. The creators of this dataset were able to make the GDPs of each country on the same 'scale' by standardizing using a World Bank indicator. Depending on the year, Gapminder had different methods on how to estimate the GDP per capita for each country. Additionally, Syria did not have any official GDP per capita estimate since 2010, so Gapminder estimated these values. This app has a multi-select dropdown where the user can select multiple countries, a range slider for the years in the dataset, and a line graph displaying the gdp per captia for each country from 1800-2100.")
,   html.Div([ # make another parent div --> for the stuff going in one row
    html.Div(
        dcc.Dropdown(
            options = [{'label': country, 'value': country} for country in data_fixed.country.unique()], # this gives the dropdown options as the diff countries in the country column of df
            id = "dropdown-country", 
            multi = True, 
            value = ['UK', 'Angola']
        ), className = "six columns",  # makes it so it takes up half (6/12)
    ), 
    html.Div(
        dcc.RangeSlider(min = 1800, max = 2100, id = 'range-slide-yr', 
                        value = [1800, 1900], 
                        marks = slider_marks, # markers for every 100 yrs 
                         tooltip = {'placement' : 'bottom', 'always_visible': True}), # makes it so that the user can see the specific year selected on slider range
        className = "six columns" # makes it so it takes up half (6/12)
    )
], className = "row"), # puts the slider & dropdown on one row 
html.Div([      # another parent div for the line graph 
    dcc.Graph(id = 'line-graph'
    )
])
])


# defining callback
@app.callback(
    Output('line-graph', 'figure'), # outputs will be the line graph 
    [Input('dropdown-country', 'value'), # inputs are the dropdowns & the range slider 
     Input('range-slide-yr', 'value')]
)
def update_fig(selected_country, selected_year): # defining function as update_fig
    dff = data_fixed[(data_fixed['year'] >= selected_year[0]) & (data_fixed['year'] <= selected_year[1]) & (data_fixed['country'].isin(selected_country))]
    # making dff which filters for the selected year min & max from the slider and the selcted countries from the dropdown ^ 
    fig = px.line(dff, x='year', # making line graph! 
                  y= 'gdp_per_capita',
                  color = 'country', title = "year vs gdp per capita per selected countries")
    fig.update_xaxes(title = "year") # titling the axes 
    fig.update_yaxes(title = 'gdp per capita')
    fig.update_layout(transition_duration=500) # doing some animation 

    return fig


In [8]:
# running app
if __name__ == '__main__':
    app.run_server(debug=True)